In [31]:
import yfinance as yf
import pandas as pd
from datetime import date, timedelta
import os 
import yaml
import time

# Step 1: Read the stock symbols from universe.csv
root_dir = os.path.dirname(os.getcwd())
# Construct the absolute path to the universe.csv file
csv_path = os.path.join(root_dir, 'data', 'universe.csv')
# Read the universe.csv file
universe_df = pd.read_csv(csv_path)
stock_symbols = universe_df['Symbol'].tolist()

# Read the threshold values from the YAML file
yaml_path = os.path.join(root_dir, 'config', 'config.yaml')
with open(yaml_path, 'r') as yaml_file:
    config = yaml.safe_load(yaml_file)
threshold_positive = config['threshold_positive']
threshold_negative = config['threshold_negative']

# Step 2: Screen stocks and calculate pivots
short_listed_stocks = {}
# Get the root directory of the project

# Loop through stock symbols
for symbol in stock_symbols[:5]:
    # Append ".NS" to symbol
    symbol_ns = symbol + ".NS"

    # Step 2.ii: Fetch data from yfinance
    end_date = date.today() - timedelta(days=1)  # Get previous day's OHLC
    start_date = end_date - timedelta(days=7)  # Get data for the last 7 days
    stock_data = yf.download(symbol_ns, start=start_date, end=end_date)

    # Step 2.iii: Calculate % return for each day
    stock_data['Prev Close'] = stock_data['Close'].shift(1)
    stock_data['% Return'] = (stock_data['Close'] - stock_data['Prev Close']) / stock_data['Prev Close'] * 100

    # Step 2.iv: Check for gain or loss and update short_listed_stocks
    if stock_data['% Return'].iloc[-1] > threshold_positive:
        short_listed_stocks[symbol] = 1
    elif stock_data['% Return'].iloc[-1] < threshold_negative:
        short_listed_stocks[symbol] = -1

    # Step 2.vi: Calculate pivots
    prev_high = stock_data['High'].iloc[-2]
    prev_low = stock_data['Low'].iloc[-2]
    prev_close = stock_data['Close'].iloc[-2]
    pivot = (prev_high + prev_low + prev_close) / 3
    res1 = 2 * pivot - prev_low
    res2 = pivot + (res1 - (2 * pivot - prev_high))
    res3 = prev_high + 2 * (pivot - prev_low)
    sup1 = 2 * pivot - prev_high
    sup2 = pivot - (res1 - sup1)
    sup3 = prev_low - 2 * (prev_high - pivot)

    # Step 2.vii: Append pivot points to short_listed_stocks
    if symbol in short_listed_stocks:
        short_listed_stocks[symbol] = {}  # Initialize the dictionary for the symbol
        short_listed_stocks[symbol]['PIVOT'] = pivot
        short_listed_stocks[symbol]['RES1'] = res1
        short_listed_stocks[symbol]['RES2'] = res2
        short_listed_stocks[symbol]['RES3'] = res3
        short_listed_stocks[symbol]['SUP1'] = sup1
        short_listed_stocks[symbol]['SUP2'] = sup2
        short_listed_stocks[symbol]['SUP3'] = sup3
    time.sleep(0.5)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [33]:
pd.DataFrame(short_listed_stocks)

,ABB,ABBOTINDIA,ADANIENT
PIVOT,4007.149984,21680.783203,2512.483317
RES1,4077.499919,21811.566406,2539.666585
RES2,4210.349935,22020.482422,2587.183268
RES3,4280.699870,22151.265625,2614.366536
SUP1,3874.299967,21471.867188,2464.966634
SUP2,3803.950033,21341.083984,2437.783366
SUP3,3671.100016,21132.167969,2390.266683


In [18]:
start_date

datetime.date(2023, 5, 27)

In [19]:
end_date

datetime.date(2023, 6, 3)

In [20]:
stock_data

,Open,High,Low,Close,Adj Close,Volume,Prev Close,% Return
Date,,,,,,,,
2023-05-29,2570.000000,2583.500000,2523.000000,2534.050049,2534.050049,2779432,NaN,NaN
2023-05-30,2529.300049,2554.149902,2475.149902,2496.449951,2496.449951,1736953,2534.050049,-1.483795
2023-05-31,2482.000000,2573.000000,2428.000000,2493.350098,2493.350098,4268439,2496.449951,-0.124170
2023-06-01,2499.899902,2560.000000,2485.300049,2492.149902,2492.149902,1704946,2493.350098,-0.048136
2023-06-02,2535.000000,2538.899902,2425.000000,2445.050049,2445.050049,5543791,2492.149902,-1.889929


In [21]:
stock_data['% Return'].iloc[-1]

-1.8899285902236378

In [42]:
import yfinance as yf
import pandas as pd
from datetime import date, timedelta
import os 
import yaml
import time
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logging.basicConfig(level=logging.ERROR,
                    format='[%(levelname)s] %(message)s (%(filename)s:%(lineno)d)',
                    handlers=[logging.StreamHandler()])
logger = logging.getLogger()
logger.showLineNumber = True

# Disable logs from yfinance module
logging.getLogger("yfinance").setLevel(logging.WARNING)

# Step 1: Read the stock symbols from universe.csv
root_dir = os.path.dirname(os.getcwd())
# Construct the absolute path to the universe.csv file
csv_path = os.path.join(root_dir, 'data', 'universe.csv')
# Read the universe.csv file
universe_df = pd.read_csv(csv_path)
stock_symbols = universe_df['Symbol'].tolist()

# Read the threshold values from the YAML file
yaml_path = os.path.join(root_dir, 'config', 'config.yaml')
with open(yaml_path, 'r') as yaml_file:
    config = yaml.safe_load(yaml_file)
threshold_positive = config['threshold_positive']
threshold_negative = config['threshold_negative']

# Step 2: Screen stocks and calculate pivots
short_listed_stocks = []
# Get the root directory of the project

# Loop through stock symbols
for symbol in stock_symbols:
    # Append ".NS" to symbol
    symbol_ns = symbol + ".NS"

    try:
        # Step 2.ii: Fetch data from yfinance
        end_date = date.today() - timedelta(days=1)  # Get previous day's OHLC
        start_date = end_date - timedelta(days=7)  # Get data for the last 7 days
        stock_data = yf.download(symbol_ns, start=start_date, end=end_date)

        # Check if data is available for the stock
        if stock_data.empty:
            logging.warning(f"No data available for stock: {symbol}")
            continue

        # Step 2.iii: Calculate % return for each day
        stock_data['Prev Close'] = stock_data['Close'].shift(1)
        stock_data['% Return'] = (stock_data['Close'] - stock_data['Prev Close']) / stock_data['Prev Close'] * 100

        # Step 2.iv: Check for gain or loss and update short_listed_stocks
        if stock_data['% Return'].iloc[-1] > threshold_positive:
            short_listed_stocks.append({'symbol': symbol, 'flag': 1, 'percent_return': stock_data['% Return'].iloc[-1]})
        elif stock_data['% Return'].iloc[-1] < threshold_negative:
            short_listed_stocks.append({'symbol': symbol, 'flag': -1, 'percent_return': stock_data['% Return'].iloc[-1]})

        # Step 2.vi: Calculate pivots
        prev_high = stock_data['High'].iloc[-2]
        prev_low = stock_data['Low'].iloc[-2]
        prev_close = stock_data['Close'].iloc[-2]
        pivot = (prev_high + prev_low + prev_close) / 3
        res1 = 2 * pivot - prev_low
        res2 = pivot + (res1 - (2 * pivot - prev_high))
        res3 = prev_high + 2 * (pivot - prev_low)
        sup1 = 2 * pivot - prev_high
        sup2 = pivot - (res1 - sup1)
        sup3 = prev_low - 2 * (prev_high - pivot)

        # Step 2.vii: Append pivot points to short_listed_stocks
        
        short_listed_stocks[-1]['PIVOT'] = pivot
        short_listed_stocks[-1]['RES1'] = res1
        short_listed_stocks[-1]['RES2'] = res2
        short_listed_stocks[-1]['RES3'] = res3
        short_listed_stocks[-1]['SUP1'] = sup1
        short_listed_stocks[-1]['SUP2'] = sup2
        short_listed_stocks[-1]['SUP3'] = sup3

        time.sleep(0.5)
    
    except Exception as e:
        logging.error(f"Error occurred for stock: {symbol}. Error message: {str(e)}")

[*********************100%***********************]  1 of 1 completed

2023-06-04 17:36:26,395 - ERROR - Error occurred for stock: ABB. Error message: list index out of range



[*********************100%***********************]  1 of 1 completed

2023-06-04 17:36:26,786 - ERROR - Error occurred for stock: ACC. Error message: list index out of range



[*********************100%***********************]  1 of 1 completed

...

2023-06-04 17:36:27,545 - ERROR - Error occurred for stock: ABBOTINDIA. Error message: list index out of range



[*********************100%***********************]  1 of 1 completed

2023-06-04 17:36:27,943 - ERROR - Error occurred for stock: ADANIENT. Error message: list index out of range



[*********************100%***********************]  1 of 1 completed

2023-06-04 17:36:28,291 - ERROR - Error occurred for stock: ADANIGREEN. Error message: list index out of range


In [38]:
pd.DataFrame(short_listed_stocks)

,symbol,flag,percent_return,PIVOT,RES1,RES2,RES3,SUP1,SUP2,SUP3
0,ABB,1,1.949480,772.400004,779.500020,789.250041,796.350057,762.649984,755.549967,745.799947
1,ABBOTINDIA,1,1.502592,21680.783203,21811.566406,22020.482422,22151.265625,21471.867188,21341.083984,21132.167969
2,ADANIENT,-1,-1.889929,2512.483317,2539.666585,2587.183268,2614.366536,2464.966634,2437.783366,2390.266683


In [48]:
import yfinance as yf
import pandas as pd
from datetime import date, timedelta
import os 
import yaml
import time
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logging.basicConfig(level=logging.ERROR,
                    format='[%(levelname)s] %(message)s (%(filename)s:%(lineno)d)',
                    handlers=[logging.StreamHandler()])
logger = logging.getLogger()
logger.showLineNumber = True

# Disable logs from yfinance module
logging.getLogger("yfinance").setLevel(logging.WARNING)

# Step 1: Read the stock symbols from universe.csv
root_dir = os.path.dirname(os.getcwd())
# Construct the absolute path to the universe.csv file
csv_path = os.path.join(root_dir, 'data', 'universe.csv')
# Read the universe.csv file
universe_df = pd.read_csv(csv_path)
stock_symbols = universe_df['Symbol'].tolist()

# Read the threshold values from the YAML file
yaml_path = os.path.join(root_dir, 'config', 'config.yaml')
with open(yaml_path, 'r') as yaml_file:
    config = yaml.safe_load(yaml_file)
threshold_positive = config['threshold_positive']
threshold_negative = config['threshold_negative']

# Step 2: Screen stocks and calculate pivots
short_listed_stocks = []
# Get the root directory of the project

# Loop through stock symbols
for symbol in stock_symbols:
    # Append ".NS" to symbol
    symbol_ns = symbol + ".NS"

    # Step 2.ii: Fetch data from yfinance
    end_date = date.today() - timedelta(days=1)  # Get previous day's OHLC
    start_date = end_date - timedelta(days=7)  # Get data for the last 7 days
    try:
        stock_data = yf.download(symbol_ns, start=start_date, end=end_date)

        # Check if data is available for the stock
        if stock_data.empty:
            logging.warning(f"No data available for stock: {symbol}")
            continue

        # Step 2.iii: Calculate % return for each day
        stock_data['Prev Close'] = stock_data['Close'].shift(1)
        stock_data['% Return'] = (stock_data['Close'] - stock_data['Prev Close']) / stock_data['Prev Close'] * 100

        # Step 2.iv: Check for gain or loss and update short_listed_stocks
        if stock_data['% Return'].iloc[-1] > threshold_positive:
            short_listed_stocks.append({'symbol': symbol, 'flag': 1, 'percent_return': stock_data['% Return'].iloc[-1]})
        elif stock_data['% Return'].iloc[-1] < threshold_negative:
            short_listed_stocks.append({'symbol': symbol, 'flag': -1, 'percent_return': stock_data['% Return'].iloc[-1]})

        # Step 2.vi: Calculate pivots
        prev_high = stock_data['High'].iloc[-2]
        prev_low = stock_data['Low'].iloc[-2]
        prev_close = stock_data['Close'].iloc[-2]
        pivot = (prev_high + prev_low + prev_close) / 3
        res1 = 2 * pivot - prev_low
        res2 = pivot + (res1 - (2 * pivot - prev_high))
        res3 = prev_high + 2 * (pivot - prev_low)
        sup1 = 2 * pivot - prev_high
        sup2 = pivot - (res1 - sup1)
        sup3 = prev_low - 2 * (prev_high - pivot)

        short_listed_stocks[symbol]['PIVOT'] = pivot
        short_listed_stocks[symbol]['RES1'] = res1
        short_listed_stocks[symbol]['RES2'] = res2
        short_listed_stocks[symbol]['RES3'] = res3
        short_listed_stocks[symbol]['SUP1'] = sup1
        short_listed_stocks[symbol]['SUP2'] = sup2
        short_listed_stocks[symbol]['SUP3'] = sup3

        stock_dict = {'symbol': symbol, 'PIVOT': pivot, 'RES1': res1, 'RES2': res2, 'RES3': res3,
                      'SUP1': sup1, 'SUP2': sup2, 'SUP3': sup3}
        short_listed_stocks.append(stock_dict)

        time.sleep(0.5)
    except Exception as e:
        logging.error(f"Error occurred for stock: {symbol}. Error message: {str(e)}")

[*********************100%***********************]  1 of 1 completed

2023-06-04 17:55:26,129 - ERROR - Error occurred for stock: NYKAA. Error message: list indices must be integers or slices, not str



[*********************100%***********************]  1 of 1 completed

2023-06-04 17:55:26,531 - ERROR - Error occurred for stock: ZOMATO. Error message: list indices must be integers or slices, not str



[*********************100%***********************]  1 of 1 completed

2023-06-04 17:55:26,938 - ERROR - Error occurred for stock: ZYDUSLIFE. Error message: list indices must be integers or slices, not str


In [47]:
short_listed_stocks

[{'symbol': 'NYKAA', 'flag': 1, 'percent_return': 7.609986807770045},
 {'symbol': 'ZOMATO', 'flag': 1, 'percent_return': 4.70935205497386}]

In [50]:
stock_data['High'].iloc[-2]

514.4000244140625

In [51]:
short_listed_stocks

[{'symbol': 'NYKAA', 'flag': 1, 'percent_return': 7.609986807770045},
 {'symbol': 'ZOMATO', 'flag': 1, 'percent_return': 4.70935205497386}]

In [64]:
import yfinance as yf
import pandas as pd
from datetime import date, timedelta
import os 
import yaml
import time
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logging.basicConfig(level=logging.ERROR,
                    format='[%(levelname)s] %(message)s (%(filename)s:%(lineno)d)',
                    handlers=[logging.StreamHandler()])
logger = logging.getLogger()
logger.showLineNumber = True

# Disable logs from yfinance module
logging.getLogger("yfinance").setLevel(logging.WARNING)

# Step 1: Read the stock symbols from universe.csv
root_dir = os.path.dirname(os.getcwd())
# Construct the absolute path to the universe.csv file
csv_path = os.path.join(root_dir, 'data', 'universe.csv')
# Read the universe.csv file
universe_df = pd.read_csv(csv_path)
stock_symbols = universe_df['Symbol'].tolist()

# Read the threshold values from the YAML file
yaml_path = os.path.join(root_dir, 'config', 'config.yaml')
with open(yaml_path, 'r') as yaml_file:
    config = yaml.safe_load(yaml_file)
threshold_positive = config['threshold_positive']
threshold_negative = config['threshold_negative']

# Step 2: Screen stocks and calculate pivots
short_listed_stocks = []
# Get the root directory of the project

# Loop through stock symbols
for symbol in stock_symbols:
    stock_dict ={}
    # Append ".NS" to symbol
    symbol_ns = symbol + ".NS"

    # Step 2.ii: Fetch data from yfinance
    end_date = date.today() - timedelta(days=1)  # Get previous day's OHLC
    start_date = end_date - timedelta(days=7)  # Get data for the last 7 days
    stock_data = yf.download(symbol_ns, start=start_date, end=end_date)

    # Check if data is available for the stock
    if stock_data.empty:
        logging.warning(f"No data available for stock: {symbol}")
        continue

    # Step 2.iii: Calculate % return for each day
    stock_data['Prev Close'] = stock_data['Close'].shift(1)
    stock_data['% Return'] = (stock_data['Close'] - stock_data['Prev Close']) / stock_data['Prev Close'] * 100

    # Step 2.iv: Check for gain or loss and update short_listed_stocks
    stock_dict['symbol'] = symbol
    stock_dict['percent_return'] = stock_data['% Return'].iloc[-1]
    if stock_data['% Return'].iloc[-1] > threshold_positive:
        stock_dict['flag'] = 1
        #short_listed_stocks.append({'symbol': symbol, 'flag': 1, 'percent_return': stock_data['% Return'].iloc[-1]})
    elif stock_data['% Return'].iloc[-1] < threshold_negative:
        #short_listed_stocks.append({'symbol': symbol, 'flag': -1, 'percent_return': stock_data['% Return'].iloc[-1]})
        stock_dict['flag'] = -1
    else:
        stock_dict['flag'] = 0

    # Step 2.vi: Calculate pivots
    prev_high = stock_data['High'].iloc[-1]
    prev_low = stock_data['Low'].iloc[-1]
    prev_close = stock_data['Close'].iloc[-1]
    pivot = (prev_high + prev_low + prev_close) / 3
    res1 = 2 * pivot - prev_low
    res2 = pivot + (res1 - (2 * pivot - prev_high))
    res3 = prev_high + 2 * (pivot - prev_low)
    sup1 = 2 * pivot - prev_high
    sup2 = pivot - (res1 - sup1)
    sup3 = prev_low - 2 * (prev_high - pivot)

    # short_listed_stocks[symbol]['PIVOT'] = pivot
    # short_listed_stocks[symbol]['RES1'] = res1
    # short_listed_stocks[symbol]['RES2'] = res2
    # short_listed_stocks[symbol]['RES3'] = res3
    # short_listed_stocks[symbol]['SUP1'] = sup1
    # short_listed_stocks[symbol]['SUP2'] = sup2
    # short_listed_stocks[symbol]['SUP3'] = sup3
    stock_dict['PIVOT'] = pivot
    stock_dict['RES1'] = res1
    stock_dict['RES2'] = res2
    stock_dict['RES3'] = res3
    stock_dict['SUP1'] = sup1
    stock_dict['SUP2'] = sup2
    stock_dict['SUP3'] = sup3

    # pivot_list = [{'PIVOT': pivot}, {'RES1': res1}, {'RES2': res2}, {'RES3': res3},
    #                 {'SUP1': sup1}, {'SUP2': sup2}, {'SUP3': sup3}]
    # stock_dict.update(pivot_list)
    short_listed_stocks.append(stock_dict)

    time.sleep(0.5)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [65]:
pd.DataFrame(short_listed_stocks)

,symbol,percent_return,flag,PIVOT,RES1,RES2,RES3,SUP1,SUP2,SUP3
0,NYKAA,7.609987,1,132.750000,139.800003,143.850006,150.900009,128.699997,121.649994,117.599991
1,ZOMATO,4.709352,1,71.283333,73.066666,74.983330,76.766663,69.366669,67.583336,65.666672
2,ZYDUSLIFE,0.895983,0,517.416667,521.433309,524.866618,528.883260,513.983358,509.966715,506.533407


In [66]:
import yfinance as yf
import pandas as pd
from datetime import date, timedelta
import os
import yaml
import time
import logging


class StockAnalyzer:
    def __init__(self):
        # Configure logging
        logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
        logging.basicConfig(level=logging.ERROR,
                            format='[%(levelname)s] %(message)s (%(filename)s:%(lineno)d)',
                            handlers=[logging.StreamHandler()])
        self.logger = logging.getLogger()
        self.logger.showLineNumber = True

        # Disable logs from yfinance module
        logging.getLogger("yfinance").setLevel(logging.WARNING)

        # Step 1: Read the stock symbols from universe.csv
        root_dir = os.path.dirname(os.getcwd())
        # Construct the absolute path to the universe.csv file
        csv_path = os.path.join(root_dir, 'data', 'universe.csv')
        # Read the universe.csv file
        self.universe_df = pd.read_csv(csv_path)
        self.stock_symbols = self.universe_df['Symbol'].tolist()

        # Read the threshold values from the YAML file
        yaml_path = os.path.join(root_dir, 'config', 'config.yaml')
        with open(yaml_path, 'r') as yaml_file:
            config = yaml.safe_load(yaml_file)
        self.threshold_positive = config['threshold_positive']
        self.threshold_negative = config['threshold_negative']

        self.short_listed_stocks = []

    def run_analysis(self):
        for symbol in self.stock_symbols:
            stock_data = self.fetch_stock_data(symbol)
            if stock_data is None:
                continue

            percent_return = self.calculate_percent_return(stock_data)
            flag = self.check_flag(percent_return)
            pivot_points = self.calculate_pivot_points(stock_data)

            stock_dict = {
                'symbol': symbol,
                'flag': flag,
                'percent_return': percent_return,
                **pivot_points
            }
            self.short_listed_stocks.append(stock_dict)
            time.sleep(0.5)

    def fetch_stock_data(self, symbol):
        symbol_ns = symbol + ".NS"
        end_date = date.today() - timedelta(days=1)
        start_date = end_date - timedelta(days=7)
        stock_data = yf.download(symbol_ns, start=start_date, end=end_date)

        if stock_data.empty:
            self.logger.warning(f"No data available for stock: {symbol}")
            return None

        return stock_data

    def calculate_percent_return(self, stock_data):
        stock_data['Prev Close'] = stock_data['Close'].shift(1)
        stock_data['% Return'] = (stock_data['Close'] - stock_data['Prev Close']) / stock_data['Prev Close'] * 100
        return stock_data['% Return'].iloc[-1]

    def check_flag(self, percent_return):
        if percent_return > self.threshold_positive:
            return 1
        elif percent_return < self.threshold_negative:
            return -1
        else:
            return 0

    def calculate_pivot_points(self, stock_data):
        prev_high = stock_data['High'].iloc[-1]
        prev_low = stock_data['Low'].iloc[-1]
        prev_close = stock_data['Close'].iloc[-1]
        pivot = (prev_high + prev_low + prev_close) / 3
        res1 = 2 * pivot - prev_low
        res2 = pivot + (res1 - (2 * pivot - prev_high))
        res3 = prev_high + 2 * (pivot - prev_low)
        sup1 = 2 * pivot - prev_high
        sup2 = pivot - (res1 - sup1)
        sup3 = prev_low - 2 * (prev_high - pivot)

        pivot_points = {
            'PIVOT': pivot,
            'RES1': res1,
            'RES2': res2,
            'RES3': res3,
            'SUP1': sup1,
            'SUP2': sup2,
            'SUP3': sup3
        }

        return pivot_points


# Create an instance of StockAnalyzer and run the analysis
analyzer = StockAnalyzer()
analyzer.run_analysis()

# Access the short-listed stocks
short_listed_stocks = analyzer.short_listed_stocks

# Convert short_listed_stocks to a pandas DataFrame
df = pd.DataFrame(short_listed_stocks)
print(df)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
      symbol  flag  percent_return       PIVOT        RES1        RES2  \
0      NYKAA     1        7.609987  132.750000  139.800003  143.850006   
1     ZOMATO     1        4.709352   71.283333   73.066666   74.983330   
2  ZYDUSLIFE     0        0.895983  517.416667  521.433309  524.866618   

         RES3        SUP1        SUP2        SUP3  
0  150.900009  128.699997  121.649994  117.599991  
1   76.766663   69.366669   67.583336   65.666672  
2  528.883260  513.983358  509.966715  506.533407  


In [67]:
df

,symbol,flag,percent_return,PIVOT,RES1,RES2,RES3,SUP1,SUP2,SUP3
0,NYKAA,1,7.609987,132.750000,139.800003,143.850006,150.900009,128.699997,121.649994,117.599991
1,ZOMATO,1,4.709352,71.283333,73.066666,74.983330,76.766663,69.366669,67.583336,65.666672
2,ZYDUSLIFE,0,0.895983,517.416667,521.433309,524.866618,528.883260,513.983358,509.966715,506.533407


In [2]:
import yfinance as yf
import pandas as pd
from datetime import date, timedelta, datetime, time
from smartapi import SmartConnect
import time
import logging
import pytz
import yaml
import pyotp

In [3]:
def load_credentials():
    with open("../config/api_credentials.yaml", "r") as file:
        credentials = yaml.safe_load(file)
    api_key = credentials["api_key"]
    secret_key = credentials["secret_key"]
    client_id = credentials["client_id"]
    password = credentials["password"]
    print(api_key, secret_key, client_id, password)
    return api_key, secret_key, client_id, password
def generate_smart_connect_object(api_key, secret_key, client_id, password ):
    obj = SmartConnect(api_key=api_key)
    totp = pyotp.TOTP(s=secret_key)
    data = obj.generateSession(client_id, password, totp.now())
    # One time script to refresh NIFTY data
    refreshToken= data['data']['refreshToken']
    #fetch the feedtoken
    feedToken=obj.getfeedToken()
    #fetch User Profile
    userProfile= obj.getProfile(refreshToken)
    print(userProfile)
    return obj
api_key, secret_key, client_id, password = load_credentials()    
obj = generate_smart_connect_object(api_key, secret_key, client_id, password)

JyXQQd0X C5O2A5URXI5LRRVMT7Y7TPYJFM VADL1002 2112
{'status': True, 'message': 'SUCCESS', 'errorcode': '', 'data': {'clientcode': 'VADL1002', 'name': 'BHAVESH GHODASARA', 'email': '', 'mobileno': '', 'exchanges': ['bse_cm', 'nse_cm', 'nse_fo', 'cde_fo', 'mcx_fo', 'ncx_fo'], 'products': ['BO', 'NRML', 'CO', 'CNC', 'MIS', 'MARGIN'], 'lastlogintime': '', 'broker': ''}}


In [4]:
print(obj)

In [5]:
ltp = obj.ltpData("NSE", f"PRESTIGE-EQ", "20302")
ltp

{'status': True,
 'message': 'SUCCESS',
 'errorcode': '',
 'data': {'exchange': 'NSE',
  'tradingsymbol': 'PRESTIGE-EQ',
  'symboltoken': '20302',
  'open': 574.15,
  'high': 581.45,
  'low': 574.05,
  'close': 574.15,
  'ltp': 580.15}}

In [11]:
ltp['data']['ltp']

565.0

In [20]:
ltp = obj.ltpData("NSE", f"PRESTIGE-EQ", "20302")

TypeError: ltpData() missing 1 required positional argument: 'symboltoken'

In [23]:
obj.getCandleData({
        "exchange": "NSE",
            "symboltoken": "3045",
            "interval": "ONE_MINUTE",
            "fromdate": "2021-02-08 09:00", 
            "todate": "2021-02-08 09:16"}
)

{'exchange': 'NFO', 'symboltoken': '7', 'interval': 'ONE_DAY', 'fromdate': '2023-05-31 00:00', 'todate': '2023-06-07 23:59'}

{'status': True,
 'message': 'SUCCESS',
 'errorcode': '',
 'data': [['2021-02-08T09:15:00+05:30', 398.0, 403.45, 397.3, 401.85, 2609877],
  ['2021-02-08T09:16:00+05:30', 402.15, 403.0, 401.45, 401.75, 1470752]]}

In [13]:
from toolkit.fileutils import Fileutils
from omspy_brokers.finvasia import Finvasia
import logging
import os
import yaml

logging.basicConfig(level=logging.DEBUG)

# sec_dir = "../../../"
with open("../config/api_credentials.yaml", "r") as file:
    credentials = yaml.safe_load(file)
m = credentials
pmart = Finvasia(user_id=m['uid'], password=m['pwd'], pin=m['factor2'],
                vendor_code=m['vc'], app_key=m['app_key'], imei="1234",
                broker="profitmart")
if pmart.authenticate():
    print("success")

token = pmart.instrument_symbol("NSE", "SBIN-EQ")
resp = pmart.scriptinfo("NSE", token)
print(resp)

DEBUG:NorenRestApiPy.NorenApi:https://profitmax.profitmart.in/NorenWClientTP/QuickAuth
DEBUG:NorenRestApiPy.NorenApi:Req:jData={"source": "API", "apkversion": "1.0.0", "uid": "35660003", "pwd": "6ad0955ebe85166b506fdc895574ef8a0db202fdba8891d1598f930b68623acf", "factor2": "AJLPD7024M", "vc": "35660003", "appkey": "472efe01e3e747afec41e45b97d066ad4fb35964b58b3280750bca17caf5769d", "imei": "1234"}
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): profitmax.profitmart.in:443
DEBUG:urllib3.connectionpool:https://profitmax.profitmart.in:443 "POST /NorenWClientTP/QuickAuth HTTP/1.1" 200 1004
DEBUG:NorenRestApiPy.NorenApi:Reply:{"request_time":"08:20:21 30-06-2023","actid":"35660003","access_type":["API","TT","WEB","MOB"],"uname":"NARESH KUMAR DAS","prarr":[{"prd":"C","s_prdt_ali":"CNC","exch":["NSE"]},{"prd":"M","s_prdt_ali":"NRML","exch":["NFO","CDS","MCX"]},{"prd":"I","s_prdt_ali":"MIS","exch":["NSE","NFO","CDS","MCX"]},{"prd":"H","s_prdt_ali":"CO","exch":["NSE","NFO","CDS","

success


DEBUG:urllib3.connectionpool:https://profitmax.profitmart.in:443 "POST /NorenWClientTP/SearchScrip HTTP/1.1" 200 147
DEBUG:NorenRestApiPy.NorenApi:{"stat":"Ok","values":[{"exch":"NSE","token":"3045","tsym":"SBIN-EQ","cname":"STATE BANK OF INDIA","instname":"EQ","pp":"2","ls":"1","ti":"0.05"}]}
DEBUG:NorenRestApiPy.NorenApi:https://profitmax.profitmart.in/NorenWClientTP/GetQuotes
DEBUG:NorenRestApiPy.NorenApi:jData={"uid": "35660003", "exch": "NSE", "token": "3045"}&jKey=090f71926578bd3b66c1cdd297a7487fd8d35780bce51e03ee058afba472793b
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): profitmax.profitmart.in:443
DEBUG:urllib3.connectionpool:https://profitmax.profitmart.in:443 "POST /NorenWClientTP/GetQuotes HTTP/1.1" 200 356
DEBUG:NorenRestApiPy.NorenApi:{"request_time":"08:20:21 30-06-2023","stat":"Ok","exch":"NSE","tsym":"SBIN-EQ","cname":"STATE BANK OF INDIA","symname":"SBIN","seg":"EQT","instname":"EQ","isin":"INE062A01020","pp":"2","ls":"1","ti":"0.05","mult":"1","uc":

{'request_time': '08:20:21 30-06-2023', 'stat': 'Ok', 'exch': 'NSE', 'tsym': 'SBIN-EQ', 'cname': 'STATE BANK OF INDIA', 'symname': 'SBIN', 'seg': 'EQT', 'instname': 'EQ', 'isin': 'INE062A01020', 'pp': '2', 'ls': '1', 'ti': '0.05', 'mult': '1', 'uc': '626.75', 'lc': '512.85', 'wk52_h': '629.54', 'wk52_l': '458.00', 'prcftr_d': '(1 / 1 ) * (1 / 1)', 'token': '3045', 'lp': '569.79', 'c': '569.79'}


In [15]:
from NorenApi import NorenApi
api = NorenApi(
    host="https://profitmax.profitmart.in/NorenWClientTP",
    websocket='wss://profitmax.profitmart.in/NorenWSTP/',
)

In [17]:
api.plac

DEBUG:NorenApi:https://profitmax.profitmart.in/NorenWClientTP/Limits


AttributeError: 'NorenApi' object has no attribute '_NorenApi__username'

In [11]:
token = pmart.instrument_symbol("NSE", "SBIN-EQ")

DEBUG:NorenRestApiPy.NorenApi:https://profitmax.profitmart.in/NorenWClientTP/SearchScrip
DEBUG:NorenRestApiPy.NorenApi:jData={"uid": "35660003", "exch": "NSE", "stext": "SBIN-EQ"}&jKey=b274fcb70bea55a953ce846a654bf046c508dd0223e225f4f4c0b3d9d1b9e8ae
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): profitmax.profitmart.in:443
DEBUG:urllib3.connectionpool:https://profitmax.profitmart.in:443 "POST /NorenWClientTP/SearchScrip HTTP/1.1" 401 65
DEBUG:NorenRestApiPy.NorenApi:{"stat":"Not_Ok","emsg":"Session Expired :  Invalid Session Key"}


TypeError: 'NoneType' object is not subscriptable

In [12]:
exchange = 'NSE'
token = '11654'
start_datetime = '1686594600'
end_datetime = '1687199399'
quote_history = pmart.historical(exchange, str(token),start_datetime ,end_datetime)

DEBUG:NorenRestApiPy.NorenApi:https://profitmax.profitmart.in/NorenWClientTP/TPSeries
DEBUG:NorenRestApiPy.NorenApi:jData={"ordersource": "API", "uid": "35660003", "exch": "NSE", "token": "11654", "st": "1686594600", "et": "1687199399"}&jKey=b274fcb70bea55a953ce846a654bf046c508dd0223e225f4f4c0b3d9d1b9e8ae
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): profitmax.profitmart.in:443
DEBUG:urllib3.connectionpool:https://profitmax.profitmart.in:443 "POST /NorenWClientTP/TPSeries HTTP/1.1" 401 65
DEBUG:NorenRestApiPy.NorenApi:{"stat":"Not_Ok","emsg":"Session Expired :  Invalid Session Key"}


In [20]:
columns=['time', 'into', 'inth', 'intl', 'intc', 'intv']
df = pd.DataFrame(quote_history, columns=columns)
logging.debug(f"Shape of dataframe is {df.shape}")
df['time'] = pd.to_datetime(df['time'], format='%d-%m-%Y %H:%M:%S')
df = df.rename(columns={'time': 'Date', 'into': 'Open', 'inth': 'High', 'intl': 'Low', 'intc': 'Close', 'intv': 'Volume'})
unique_dates = df['Date'].dt.date.unique()
df = df.set_index('Date')
for c in ['Open','High','Low','Close']:
    df[c] = df[c].astype('float')
df['Volume'] = df['Volume'].astype('int')    
df_daily = df.resample('D').agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'})
df_daily = df_daily.reset_index()
df_daily['Date'] = pd.to_datetime(df_daily['Date'])
df_daily = df_daily[df_daily['Date'].dt.date.isin(unique_dates)]

DEBUG:root:Shape of dataframe is (1960, 6)


In [27]:
# Set the parameters.
buy_or_sell = "B"
product_type = "B"
exchange = "NSE"
tradingsymbol = "SBIN"
quantity = 1
discloseqty = 0
price_type = "SL-L"
price = 570
trigger_price = 569.5
stoploss = 550
target = 590
validity ='DAY'
pmart.order_place(side=buy_or_sell, product=product_type,
                  exchange=exchange, symbol=tradingsymbol, 
                  quantity=quantity, order_type=price_type,
                  validity=validity, 
                  price=price, trigger_price=trigger_price,
                  bookloss_price=stoploss, bookprofit_price=target
                  )

DEBUG:NorenRestApiPy.NorenApi:https://profitmax.profitmart.in/NorenWClientTP/PlaceOrder
DEBUG:NorenRestApiPy.NorenApi:jData={"ordersource": "API", "uid": "35660003", "actid": "35660003", "trantype": "B", "prd": "B", "exch": "NSE", "tsym": "SBIN-EQ", "qty": "1", "dscqty": "0", "prctyp": "SL-LMT", "prc": "570", "trgprc": "569.5", "ret": "DAY", "remarks": null, "amo": "NO", "blprc": "550", "bpprc": "590"}&jKey=0ba4ee4d3784e3b92d61697395ad86803f5005a590acb3289385ad011ca756b9
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): profitmax.profitmart.in:443
DEBUG:urllib3.connectionpool:https://profitmax.profitmart.in:443 "POST /NorenWClientTP/PlaceOrder HTTP/1.1" 401 65
DEBUG:NorenRestApiPy.NorenApi:{"stat":"Not_Ok","emsg":"Session Expired :  Invalid Session Key"}


AttributeError: 'NoneType' object has no attribute 'get'

In [26]:
pmart

In [ ]:
ret = api.place_order(buy_or_sell='B', product_type='C',
                        exchange='NSE', tradingsymbol='CANBK-EQ', 
                        quantity=1, discloseqty=0,price_type='SL-LMT', price=200.00, trigger_price=199.50,
                        retention='DAY', remarks='my_order_001')

In [21]:
df_daily

,Date,Open,High,Low,Close,Volume
0,2023-06-13,1955.00,2009.85,1962.0,1987.65,161519
1,2023-06-14,2004.00,2008.75,1973.0,1997.30,411010
2,2023-06-15,2009.95,2020.85,1990.0,1994.40,366350
3,2023-06-16,1980.00,2035.00,1992.5,2026.45,528174
6,2023-06-19,2051.00,2142.40,2035.0,2121.15,1266048


In [ ]:
# 2026.45 - 2121.15 = 94.7
# 4.67%

In [18]:
def getEpoch(datetimeObj = None):
    # This method converts given datetimeObj to epoch seconds
    if datetimeObj == None:
        datetimeObj = datetime.now()
    epochSeconds = datetime.timestamp(datetimeObj)
    return int(epochSeconds) # converting double to long     

In [27]:
start_date

datetime.date(2023, 6, 9)

In [38]:
from datetime import date, timedelta, datetime, time

current_time = datetime.now().time()
end_date = date.today()

start_date = end_date - timedelta(days=7)  # 7 days before the end date
start_datetime = datetime.combine(start_date, time.min)
end_datetime = datetime.combine(end_date, time.max)

if current_time < time(10, 15):
    end_datetime -= timedelta(days=1)  # Yesterday's date
elif current_time >= time(16, 0):
    end_datetime = datetime.combine(end_date, time.max)  # Today's date


DEBUG:NorenRestApiPy.NorenApi:https://profitmax.profitmart.in/NorenWClientTP/TPSeries
DEBUG:NorenRestApiPy.NorenApi:jData={"ordersource": "API", "uid": "35660003", "exch": "NSE", "token": "3456", "st": "1686249000", "et": "1686940199"}&jKey=c6682c555a69515608eebf95ff37114e5ce17a9647bd02b894722e964e83f3e5
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): profitmax.profitmart.in:443
DEBUG:urllib3.connectionpool:https://profitmax.profitmart.in:443 "POST /NorenWClientTP/TPSeries HTTP/1.1" 200 447504
DEBUG:NorenRestApiPy.NorenApi:[{"stat":"Ok","time":"16-06-2023 14:22:00","ssboe":"1686905520","into":"569.40","inth":"569.45","intl":"569.00","intc":"569.45","intvwap":"568.72","intv":"19033","intoi":"0","v":"5313714","oi":"0"},{"stat":"Ok","time":"16-06-2023 14:21:00","ssboe":"1686905460","into":"569.00","inth":"569.50","intl":"568.80","intc":"569.35","intvwap":"570.47","intv":"30035","intoi":"0","v":"5294681","oi":"0"},{"stat":"Ok","time":"16-06-2023 14:20:00","ssboe":"168690540

In [101]:
data = pmart.historical('NSE','3456',getEpoch(start_datetime),getEpoch(end_datetime))
df = pd.DataFrame(data, columns=['time', 'into', 'inth', 'intl', 'intc', 'intv'])
# df = df[['time','into','inth','intl','intc','intv']]
df['time'] = pd.to_datetime(df['time'], format='%d-%m-%Y %H:%M:%S')
df = df.rename(columns={'time': 'Date', 'into': 'Open', 'inth': 'High', 'intl': 'Low', 'intc': 'Close', 'intv': 'Volume'})
unique_dates = df['Date'].dt.date.unique()
df = df.set_index('Date')
for c in ['Open','High','Low','Close']:
    df[c] = df[c].astype('float')
df['Volume'] = df['Volume'].astype('int')    
df_daily = df.resample('D').agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'})
df_daily = df_daily.reset_index()
df_daily['Date'] = pd.to_datetime(df_daily['Date'])
df_daily = df_daily[df_daily['Date'].dt.date.isin(unique_dates)]

DEBUG:NorenRestApiPy.NorenApi:https://profitmax.profitmart.in/NorenWClientTP/TPSeries
DEBUG:NorenRestApiPy.NorenApi:jData={"ordersource": "API", "uid": "35660003", "exch": "NSE", "token": "3456", "st": "1686249000", "et": "1686940199"}&jKey=c6682c555a69515608eebf95ff37114e5ce17a9647bd02b894722e964e83f3e5
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): profitmax.profitmart.in:443


DEBUG:urllib3.connectionpool:https://profitmax.profitmart.in:443 "POST /NorenWClientTP/TPSeries HTTP/1.1" 200 461090
DEBUG:NorenRestApiPy.NorenApi:[{"stat":"Ok","time":"16-06-2023 15:31:07","ssboe":"1686909667","into":"569.80","inth":"569.80","intl":"569.80","intc":"569.80","intvwap":"0.00","intv":"0","intoi":"0","v":"8733558","oi":"0"},{"stat":"Ok","time":"16-06-2023 15:30:26","ssboe":"1686909626","into":"569.85","inth":"569.85","intl":"569.85","intc":"569.85","intvwap":"0.00","intv":"0","intoi":"0","v":"8733558","oi":"0"},{"stat":"Ok","time":"16-06-2023 15:29:00","ssboe":"1686909540","into":"569.90","inth":"570.00","intl":"569.00","intc":"569.85","intvwap":"569.09","intv":"36227","intoi":"0","v":"8733558","oi":"0"},{"stat":"Ok","time":"16-06-2023 15:28:00","ssboe":"1686909480","into":"569.45","inth":"570.00","intl":"569.20","intc":"570.00","intvwap":"569.09","intv":"54851","intoi":"0","v":"8697331","oi":"0"},{"stat":"Ok","time":"16-06-2023 15:27:00","ssboe":"1686909420","into":"569.6

In [110]:
data = pmart.historical('NSE','3456',getEpoch(start_datetime),getEpoch(end_datetime))
columns=['time', 'into', 'inth', 'intl', 'intc', 'intv']
dtypes = {'into': float, 'inth': float, 'intl': float, 'intc': float, 'intv': int}
df = pd.DataFrame(data, columns=columns, dtype=dtypes)
df = df.rename(columns={'time': 'Date', 'into': 'Open', 'inth': 'High', 'intl': 'Low', 'intc': 'Close', 'intv': 'Volume'})
unique_dates = df['Date'].dt.date.unique()
df = df.set_index('Date')
df_daily = df.resample('D').agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'})
df_daily = df_daily.reset_index()
df_daily['Date'] = pd.to_datetime(df_daily['Date'])
df_daily = df_daily[df_daily['Date'].dt.date.isin(unique_dates)]

DEBUG:NorenRestApiPy.NorenApi:https://profitmax.profitmart.in/NorenWClientTP/TPSeries
DEBUG:NorenRestApiPy.NorenApi:jData={"ordersource": "API", "uid": "35660003", "exch": "NSE", "token": "3456", "st": "1686249000", "et": "1686940199"}&jKey=c6682c555a69515608eebf95ff37114e5ce17a9647bd02b894722e964e83f3e5
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): profitmax.profitmart.in:443
DEBUG:urllib3.connectionpool:https://profitmax.profitmart.in:443 "POST /NorenWClientTP/TPSeries HTTP/1.1" 200 461090
DEBUG:NorenRestApiPy.NorenApi:[{"stat":"Ok","time":"16-06-2023 15:31:07","ssboe":"1686909667","into":"569.80","inth":"569.80","intl":"569.80","intc":"569.80","intvwap":"0.00","intv":"0","intoi":"0","v":"8733558","oi":"0"},{"stat":"Ok","time":"16-06-2023 15:30:26","ssboe":"1686909626","into":"569.85","inth":"569.85","intl":"569.85","intc":"569.85","intvwap":"0.00","intv":"0","intoi":"0","v":"8733558","oi":"0"},{"stat":"Ok","time":"16-06-2023 15:29:00","ssboe":"1686909540","into":"5

TypeError: object of type 'type' has no len()

In [102]:
data = pmart.historical('NSE', '3456', getEpoch(start_datetime), getEpoch(end_datetime))
df = pd.DataFrame(data, columns=['time', 'into', 'inth', 'intl', 'intc', 'intv'])
df = df.rename(columns={'time': 'Date', 'into': 'Open', 'inth': 'High', 'intl': 'Low', 'intc': 'Close', 'intv': 'Volume'})
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y %H:%M:%S')

df_daily = (
    df.set_index('Date')
    .astype({'Open': float, 'High': float, 'Low': float, 'Close': float, 'Volume': int})
    .resample('D')
    .agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'})
    .reset_index()
    .loc[lambda x: x['Date'].dt.date.isin(x['Date'].dt.date.unique())]
)

DEBUG:NorenRestApiPy.NorenApi:https://profitmax.profitmart.in/NorenWClientTP/TPSeries
DEBUG:NorenRestApiPy.NorenApi:jData={"ordersource": "API", "uid": "35660003", "exch": "NSE", "token": "3456", "st": "1686249000", "et": "1686940199"}&jKey=c6682c555a69515608eebf95ff37114e5ce17a9647bd02b894722e964e83f3e5
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): profitmax.profitmart.in:443
DEBUG:urllib3.connectionpool:https://profitmax.profitmart.in:443 "POST /NorenWClientTP/TPSeries HTTP/1.1" 200 461090
DEBUG:NorenRestApiPy.NorenApi:[{"stat":"Ok","time":"16-06-2023 15:31:07","ssboe":"1686909667","into":"569.80","inth":"569.80","intl":"569.80","intc":"569.80","intvwap":"0.00","intv":"0","intoi":"0","v":"8733558","oi":"0"},{"stat":"Ok","time":"16-06-2023 15:30:26","ssboe":"1686909626","into":"569.85","inth":"569.85","intl":"569.85","intc":"569.85","intvwap":"0.00","intv":"0","intoi":"0","v":"8733558","oi":"0"},{"stat":"Ok","time":"16-06-2023 15:29:00","ssboe":"1686909540","into":"5

In [111]:
data = pmart.historical('NSE','3456',getEpoch(start_datetime),getEpoch(end_datetime))
df = pd.DataFrame(data, columns=['time', 'into', 'inth', 'intl', 'intc', 'intv'])
# df = df[['time','into','inth','intl','intc','intv']]
df['time'] = pd.to_datetime(df['time'], format='%d-%m-%Y %H:%M:%S')

DEBUG:NorenRestApiPy.NorenApi:https://profitmax.profitmart.in/NorenWClientTP/TPSeries
DEBUG:NorenRestApiPy.NorenApi:jData={"ordersource": "API", "uid": "35660003", "exch": "NSE", "token": "3456", "st": "1686249000", "et": "1686940199"}&jKey=c6682c555a69515608eebf95ff37114e5ce17a9647bd02b894722e964e83f3e5
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): profitmax.profitmart.in:443
DEBUG:urllib3.connectionpool:https://profitmax.profitmart.in:443 "POST /NorenWClientTP/TPSeries HTTP/1.1" 200 461090
DEBUG:NorenRestApiPy.NorenApi:[{"stat":"Ok","time":"16-06-2023 15:31:07","ssboe":"1686909667","into":"569.80","inth":"569.80","intl":"569.80","intc":"569.80","intvwap":"0.00","intv":"0","intoi":"0","v":"8733558","oi":"0"},{"stat":"Ok","time":"16-06-2023 15:30:26","ssboe":"1686909626","into":"569.85","inth":"569.85","intl":"569.85","intc":"569.85","intvwap":"0.00","intv":"0","intoi":"0","v":"8733558","oi":"0"},{"stat":"Ok","time":"16-06-2023 15:29:00","ssboe":"1686909540","into":"5

In [113]:
df = df.set_index('time')

In [114]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2345 entries, 2023-06-16 15:31:07 to 2023-06-09 09:00:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   into    2345 non-null   object
 1   inth    2345 non-null   object
 2   intl    2345 non-null   object
 3   intc    2345 non-null   object
 4   intv    2345 non-null   object
dtypes: object(5)
memory usage: 109.9+ KB


In [ ]:
df = df.rename(columns={'time': 'Date', 'into': 'Open', 'inth': 'High', 'intl': 'Low', 'intc': 'Close', 'intv': 'Volume'})
unique_dates = df['Date'].dt.date.unique()
df = df.set_index('Date')
for c in ['Open','High','Low','Close']:
    df[c] = df[c].astype('float')
df['Volume'] = df['Volume'].astype('int')    
df_daily = df.resample('D').agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'})
df_daily = df_daily.reset_index()
df_daily['Date'] = pd.to_datetime(df_daily['Date'])
df_daily = df_daily[df_daily['Date'].dt.date.isin(unique_dates)]

In [1]:
import os, sys
#sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
from api_helper import NorenApiPy
import logging
import yaml

#enable dbug to see request and responses
logging.basicConfig(level=logging.DEBUG)

#start of our program
api = NorenApiPy()

#credentials
with open('/home/ubuntu/workspace/daily_price_series/NorenApi-Py/cred.yml') as f:
    cred = yaml.load(f, Loader=yaml.FullLoader)
    print(cred)

ret = api.login(userid = cred['user'], password = cred['pwd'], twoFA=cred['factor2'], vendor_code=cred['vc'], api_secret=cred['apikey'], imei=cred['imei'])

ret =api.get_daily_price_series(exchange="NSE",tradingsymbol="ACC-EQ",startdate="1667297289",enddate="1670231374")
print(ret)

ModuleNotFoundError: No module named 'api_helper'

In [123]:
from api_helper import NorenApiPy

ModuleNotFoundError: No module named 'api_helper'

In [124]:
from NorenRestApiPy.NorenApi import NorenApi

In [125]:
ret =api.get_daily_price_series(exchange="NSE",tradingsymbol="PAYTM-EQ",startdate="457401600",enddate="480556800")


NameError: name 'api' is not defined

In [128]:
c = Controller()

In [129]:
c.load_credentials()

INFO:root:Loading credentials.


FileNotFoundError: [Errno 2] No such file or directory: 'config/api_credentials.yaml'

In [134]:
from api_helper import NorenApiPy

ModuleNotFoundError: No module named 'api_helper'

In [135]:
import NorenRestApiPy
# ret = api.login(userid=uid, password=pwd, twoFA=factor2,
#                 vendor_code=vc, api_secret=app_key, imei=imei)

In [142]:
from ..api_helper import NorenApiPy

ImportError: attempted relative import with no known parent package

In [131]:
api = NorenApiPy()
ret = api.login(userid=uid, password=pwd, twoFA=factor2,
                vendor_code=vc, api_secret=app_key)

NameError: name 'NorenApiPy' is not defined

In [143]:
from ..api_helper import NorenApiPy

ImportError: attempted relative import with no known parent package

In [144]:
import sys
sys.path.append("../../")
from api_helper import ApiHelper

ModuleNotFoundError: No module named 'api_helper'

In [146]:
from ..api_helper import NorenApiPy

ImportError: attempted relative import with no known parent package

In [59]:
import sys
from datetime import date, timedelta, datetime
sys.path.append("..")
from api_helper import NorenApiPy
import pandas as pd

In [60]:
uid = "35660003"
pwd = "Ret2213@"
factor2 = "AJLPD7024M"
vc = "35660003"
app_key = "pn39GKbQH3quTF8MQNQb395s4G6Dy8yN"
api = NorenApiPy()
ret = api.login(userid=uid, password=pwd, twoFA=factor2,
                vendor_code=vc, api_secret=app_key, imei='123')

DEBUG:NorenRestApiPy.NorenApi:https://profitmax.profitmart.in/NorenWClientTP/QuickAuth
DEBUG:NorenRestApiPy.NorenApi:Req:jData={"source": "API", "apkversion": "1.0.0", "uid": "35660003", "pwd": "6ad0955ebe85166b506fdc895574ef8a0db202fdba8891d1598f930b68623acf", "factor2": "AJLPD7024M", "vc": "35660003", "appkey": "472efe01e3e747afec41e45b97d066ad4fb35964b58b3280750bca17caf5769d", "imei": "123"}
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): profitmax.profitmart.in:443
DEBUG:urllib3.connectionpool:https://profitmax.profitmart.in:443 "POST /NorenWClientTP/QuickAuth HTTP/1.1" 200 1004
DEBUG:NorenRestApiPy.NorenApi:Reply:{"request_time":"10:38:47 27-06-2023","actid":"35660003","access_type":["API","TT","WEB","MOB"],"uname":"NARESH KUMAR DAS","prarr":[{"prd":"C","s_prdt_ali":"CNC","exch":["NSE"]},{"prd":"M","s_prdt_ali":"NRML","exch":["NFO","CDS","MCX"]},{"prd":"I","s_prdt_ali":"MIS","exch":["NSE","NFO","CDS","MCX"]},{"prd":"H","s_prdt_ali":"CO","exch":["NSE","NFO","CDS","M

In [62]:
api.place_gtt_oco_mkt_order()

AttributeError: 'NorenApiPy' object has no attribute 'place_gtt_oco_mkt_order'

In [55]:
api.place_gtt_oco_mkt_order("SBIN", "NSE", 570,560,"B","I",1,remarks="PLACE_OCO_MKT")

AttributeError: 'NorenApiPy' object has no attribute 'place_gtt_oco_mkt_order'

In [48]:
lastBusDay = datetime(2023, 6, 22, 0, 0)
lastBusDay = lastBusDay.replace(hour=0, minute=0, second=0, microsecond=0)
enddate = int(datetime(2023, 6, 20, 0, 0).timestamp())
ret = api.get_time_price_series(exchange='NSE', token='1515', starttime=enddate, endtime=lastBusDay.timestamp(),interval=15)

In [49]:
pd.DataFrame(ret)

,stat,time,ssboe,into,inth,intl,intc,intvwap,intv,intoi,v,oi
0,Ok,21-06-2023 15:15:00,1687340700,230.80,233.40,230.80,233.20,232.12,1054592,0,4976579,0
1,Ok,21-06-2023 15:00:00,1687339800,229.00,230.85,228.75,230.85,230.03,427124,0,3921987,0
2,Ok,21-06-2023 14:45:00,1687338900,228.75,229.45,228.10,228.90,228.68,274111,0,3494863,0
3,Ok,21-06-2023 14:30:00,1687338000,227.40,229.15,227.30,228.50,228.44,282616,0,3220752,0
4,Ok,21-06-2023 14:15:00,1687337100,228.55,229.10,227.35,227.45,228.37,167591,0,2938136,0
5,Ok,21-06-2023 14:00:00,1687336200,227.90,228.65,227.35,228.45,228.19,116514,0,2770545,0
6,Ok,21-06-2023 13:45:00,1687335300,228.25,228.50,227.40,227.95,228.06,113101,0,2654031,0
7,Ok,21-06-2023 13:30:00,1687334400,226.65,228.75,226.65,228.05,227.92,210928,0,2540930,0
8,Ok,21-06-2023 13:15:00,1687333500,227.20,227.20,226.55,226.90,227.13,56020,0,2330002,0
9,Ok,21-06-2023 13:00:00,1687332600,227.30,227.35,226.80,227.05,226.99,72421,0,2273982,0


,stat,time,ssboe,into,inth,intl,intc,intvwap,intv,intoi,v,oi
0,Ok,21-06-2023 15:25:00,1687341300,232.00,233.20,231.80,233.20,232.35,237657,0,4976579,0
1,Ok,21-06-2023 15:20:00,1687341000,232.65,233.40,231.85,231.95,232.74,318316,0,4738922,0
2,Ok,21-06-2023 15:15:00,1687340700,230.80,232.80,230.80,232.80,231.63,498619,0,4420606,0
3,Ok,21-06-2023 15:10:00,1687340400,229.85,230.85,229.70,230.85,230.31,233972,0,3921987,0
4,Ok,21-06-2023 15:05:00,1687340100,229.10,229.90,229.10,229.80,229.63,152760,0,3688015,0
...,...,...,...,...,...,...,...,...,...,...,...,...
304,Ok,16-06-2023 09:20:00,1686887400,225.30,225.65,224.55,225.35,225.01,58205,0,164494,0
305,Ok,16-06-2023 09:15:00,1686887100,223.95,225.70,223.05,225.20,225.13,96725,0,106289,0
306,Ok,16-06-2023 09:10:00,1686886800,224.65,224.65,224.65,224.65,0.00,0,0,9564,0
307,Ok,16-06-2023 09:05:00,1686886500,222.90,224.65,222.90,224.65,0.00,5346,0,9564,0


In [ ]:
startdate = int(datetime(2023, 6, 16, 0, 0).timestamp())
enddate = int(datetime(2023, 6, 21, 0, 0).timestamp())
ret =api.get_daily_price_series(exchange="NSE",tradingsymbol="PAYTM-EQ",startdate=startdate,enddate=enddate)

DEBUG:NorenRestApiPy.NorenApi:
DEBUG:NorenRestApiPy.NorenApi:{"uid": "35660003", "sym": "NSE:PAYTM-EQ", "from": "1686335400", "to": "1686853800"}


MissingSchema: Invalid URL '': No scheme supplied. Perhaps you meant http://?

In [172]:
int(startdate.timestamp())

1686335400

In [190]:
import os
import logging
import pandas as pd
from utils.utils import Utils
import sys
        
today_date = pd.Timestamp.now().strftime('%Y%m%d')
# file_path = os.path.join('data', f'rocket_shortlisted_{today_date}.csv')
file_path = r"C:\Users\bhave\OneDrive\Documents\Projects\Commercial\Rocket Strategy\data\rocket_shortlisted_20230618.csv"
if os.path.isfile(file_path):
    df = pd.read_csv(file_path)
    if df.shape[0] == 0:
        logging.info("No stocks for Rocket Strategy. Exiting the code.")
        sys.exit()
    else:
        rocket_stocks = df.to_dict('records')
else:
    logging.info("No path exists")        

In [191]:
rocket_stocks

[{'Stock': 'HDFCLIFE', 'Signal': 'sell'}, {'Stock': 'ITC', 'Signal': 'buy'}]

In [193]:
rocket_stocks['Stock']

TypeError: list indices must be integers or slices, not str

In [ ]:
start_datetime = datetime.combine(start_date, dt_time.min)
end_datetime = datetime.combine(end_date, dt_time.max)
if current_time < dt_time(15, 31):
    end_datetime -= timedelta(days=1)  # Yesterday's date
elif current_time >= dt_time(16, 0):
    end_datetime = datetime.combine(end_date, dt_time.max)  # Today's date        
exchange = "NSE"
start_datetime = self.getEpoch(start_datetime)
end_datetime = self.getEpoch(end_datetime)
try:


In [ ]:
exchange = 'NSE'
token = str(7)
start_datetime = '1686594600'
end_datetime = '1687199399'
quote_history = self.obj.historical(exchange, str(token),start_datetime ,end_datetime)

In [2]:
023-06-20 09:26:33: NSE
2023-06-20 09:26:33: 7
2023-06-20 09:26:33: 1686594600
2023-06-20 09:26:33: 1687199399

SyntaxError: EOL while scanning string literal (<ipython-input-2-9b2265c1c7bf>, line 1)

In [5]:
import os
import pandas as pd
from datetime import datetime, timedelta,  time as dt_time
import time

import yaml
from utils.utils import Utils
import logging    
def read_screened_stocks():
    now = datetime.now()
    cutoff_time = datetime.combine(now.date(), dt_time(15, 30))
    print(now)
    print(cutoff_time)
    if now < cutoff_time:
        print("I am here.")
        date_str = (now - timedelta(days=1)).strftime("%Y_%m_%d")
    else:
        date_str = now.strftime("%Y_%m_%d")        
    filename = os.path.join("data", f"screened_stocks_{date_str}.csv")
    df = pd.read_csv(filename)
    screened_stocks = df.to_dict('records')
    return screened_stocks

In [6]:
read_screened_stocks()

2023-06-20 08:43:29.679214
2023-06-20 15:30:00
I am here.


FileNotFoundError: [Errno 2] No such file or directory: 'data\\screened_stocks_2023_06_19.csv'